In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Define a residual block with a matching shortcut
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x  # Save input for skip connection

    # If the number of filters is different, apply a 1x1 convolution to the shortcut
    if x.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=stride, padding="same")(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    # First convolution layer
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Second convolution layer
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Add the shortcut (skip connection)
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    
    return x

# Build the ResNet model
def build_resnet(input_shape, num_classes=10):
    inputs = layers.Input(shape=input_shape)

    # Initial convolution layer
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding="same")(x)

    # Add residual blocks
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = residual_block(x, 256)
    x = residual_block(x, 256)

    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Fully connected output layer
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    # Create the model
    model = models.Model(inputs, outputs)
    return model

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have a single channel (grayscale) and pad to 32x32
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))
x_train = tf.image.resize(x_train, (32, 32))
x_test = tf.image.resize(x_test, (32, 32))

# Normalize pixel values to the range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build the model
model = build_resnet(input_shape=(32, 32, 1), num_classes=10)

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=15, batch_size=64, validation_data=(x_test, y_test))

# save the model
model.save('resnet_model.h5')

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")



Epoch 1/15
938/938 [==============================] - 34s 30ms/step - loss: 0.0998 - accuracy: 0.9691 - val_loss: 0.1856 - val_accuracy: 0.9487
Epoch 2/15
938/938 [==============================] - 27s 29ms/step - loss: 0.0438 - accuracy: 0.9860 - val_loss: 0.0782 - val_accuracy: 0.9762
Epoch 3/15
938/938 [==============================] - 27s 29ms/step - loss: 0.0358 - accuracy: 0.9888 - val_loss: 0.0804 - val_accuracy: 0.9737
Epoch 4/15
938/938 [==============================] - 27s 29ms/step - loss: 0.0299 - accuracy: 0.9906 - val_loss: 0.0460 - val_accuracy: 0.9859
Epoch 5/15
938/938 [==============================] - 27s 29ms/step - loss: 0.0259 - accuracy: 0.9921 - val_loss: 0.0327 - val_accuracy: 0.9901
Epoch 6/15
938/938 [==============================] - 28s 29ms/step - loss: 0.0247 - accuracy: 0.9925 - val_loss: 0.0328 - val_accuracy: 0.9907
Epoch 7/15
938/938 [==============================] - 28s 30ms/step - loss: 0.0211 - accuracy: 0.9934 - val_loss: 0.0505 - val_accuracy: